In [1]:
from pathlib import Path
from gensim.models import KeyedVectors
from collections import Counter
import json 
import re, string
import pickle
import numpy as np
import random
import mygrad as mg

class Coco:
    punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
    
    def strip_punc(self, corpus):
        """ 
        Removes all punctuation from a string.

        Parameters
        ----------
        corpus : str

        Returns
        -------
        str
            the corpus with all punctuation removed
        """
        # substitute all punctuation marks with ""
        
        return self.punc_regex.sub('', str(corpus))
    
    def to_counter(self, doc):
        """ 
        Produce word-count of document, removing all punctuation
        and making all the characters lower-cased.
        
        Parameters
        ----------
        doc : str
        
        Returns
        -------
        collections.Counter
            lower-cased word -> count
        """
        return Counter(self.strip_punc(doc).lower().split())

    def to_vocab(self, counters, k=None):
        """ 
        Convert a collection of counters to a sorted list of the top-k most common words 
        
        Parameters
        ----------
        counters : Sequence[collections.Counter]
            A list of counters; each one is a word tally for a document
        
        k : Optional[int]
            If specified, only the top-k words are returned
            
        Returns
        -------
        List[str]
            A sorted list of the unique strings.
        """
        vocab = Counter()
        for counter in counters:
            vocab.update(counter)
            
        return sorted(i for i,j in vocab.most_common(k))

    def __init__(self): 
        """
        load COCO metadata (json file ["images"] ["annotations"])
        load glove data (dictionary {word : word_embedding})
        load in resnet data from resnet18_features.pkl (dictionary {img id : dvector})
        
        initialize the following attributes:
        image-ID -> [cap-ID-1, cap-ID-2, ...]
        caption-ID -> image-ID
        caption-ID -> caption (e.g. 24 -> "two dogs on the grass")
        
        initialize vocab list and counters list as attributes
       
        Parameters
        ----------
        None

        Returns
        -------
        None
        """
        # load COCO metadata
        with Path(r"C:\Users\nicho\Downloads\captions_train2014.json").open() as f:
            self.coco_data = json.load(f)
        
        # load GloVe-200 embeddings
        self.glove = KeyedVectors.load_word2vec_format(r"C:\Users\nicho\Downloads\glove.6B.200d.txt.w2v", binary=False)

        # load image descriptor vectors
        with Path(r"C:\Users\nicho\Downloads\resnet18_features.pkl").open('rb') as f:
            self.resnet18_features = pickle.load(f)
        
        self.imgid_to_capid = {}
        self.capid_to_imgid = {}
        self.capid_to_capstr = {}

        for caption in self.coco_data["annotations"]:
            # input caption_id to imgid_to_capid if the img_id key exists
            if self.imgid_to_capid.__contains__(caption["image_id"]):
                self.imgid_to_capid[caption["image_id"]].append(caption["id"])
            # else create new img_id object and create new caption_id list
            else:
                self.imgid_to_capid[caption["image_id"]] = [caption["id"]]

            # input img_id to capid_to_imgid
            self.capid_to_imgid[caption["id"]] = caption["image_id"]
            # input caption to capid_to_capstr
            self.capid_to_capstr[caption["id"]] = caption["caption"]
    
            # COMPUTING THE IDF
            # calculate the total number of captions
                N = len(self.coco_data["annotations"])
            # get all the words among all the captions
                cap_counter = caption.tocounter
                for t in cap_counter
                append t to vocab
            # get the number of captions that word t appears in
                
        self.vocab = self.to_vocab(self.counters)
            
    def random_pair(self):
        """
        returns a random caption_string and respective image id

        Parameters
        ---------
        None
        
        Returns 
        -------
        Tuple(int, string)
            this contains the random caption_string and respective image id

        """    
        # random respective caption string
        captions = self.coco_data["annotations"]
        i = random.randint(0, len(captions)) 
        
        caption_info = captions[i]
        
        return caption_info["image_id"], caption_info["caption"]
        
    def vectorize_image(self, image_id):
        """
        takes in an image_id and returns the descriptor vector of the image

        Parameters
        ---------
        image_id: int
            unique integer ID for the image in coco_data
        
        Returns 
        -------
        image_dvector: np.array shape-(512,)
            a descriptor vector of the image as provided by RESNET
        """
        if image_id not in self.resnet18_features.keys():
            return np.zeros((512,))
        else:
            return self.resnet18_features[image_id]
    
    """
    def get_idf(self):
    """
        """ 
        Given the vocabulary, and the word-counts for each document, computes
        the inverse document frequency (IDF) for each term in the vocabulary.
        
        Parameters
        ----------
        None
        
        Returns
        -------
        nt: dict{string, float}
            A dictionary storing the IDF for each term in vocab
        """
    """
        nt = {}
        N = len(self.counters)
        for t in self.vocab:
            total = 0
            for counter in self.counters:
                if t in counter:
                    total += 1

            nt[t] = np.log10(N / total)

        return nt
    """
    
    """
    def embed_text(self, text_string):
    """
        """
        returns normal_text_embedding

        Parameters
        ---------
        text_string: String
            a caption/query text 
        
        Returns 
        -------
        String
            normal text embedding
        """
    """
        # returns normal_text_embedding
        # embed any caption / query text using GloVe-200 embeddings weighted by IDFs of words across captions (pass in either a user's query or existing caption)     
        # lowercase, remove punc, tokenize
        text_string = self.strip_punc(text_string).lower().split()

        text_embedding = []
        idf = self.get_idf()
        for word in text_string:
        # check if each word in given string is in glove[], if not then embedding vector 0
        # else get glove vector (200,) for it
            glove_vector = 0
            if word in self.glove:
                glove_vector = self.glove[word]
            
            idf_word = idf[word]
            text_embedding.append(glove_vector * idf_word)

        # add all together for the final phrase embed vector, then normalize
        normal_text_embedding = mg.sqrt(mg.einsum("ij, ij -> i", text_embedding, text_embedding)).reshape(-1, 1)

        # return normal_text_embedding
        return normal_text_embedding
    """
    
    def get_data(self):
        """
        returns coco_data, glove , resnet18_features, imgid_to_capid, capid_to_imgid, capid_to_capstr, counters

        Parameters
        ---------
        None

        Returns 
        -------
        Tuple(dict, dict, dict)
            contains coco data, glove data, and resnet data
        """
        return (self.coco_data, self.glove, self.resnet18_features, self.imgid_to_capid, self.capid_to_imgid, self.capid_to_capstr, self.counters)

C:\Users\nicho\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# INITIALIZE COCO, GLOVE, RESNET DATA
coco = Coco()

In [3]:
# RETRIEVE DICTIONARIES W/ VARIOUS MAPPINGS
coco_data, glove, resnet18_features, imgid_to_capid, capid_to_imgid, capid_to_capstr, counters = coco.get_data()

In [4]:
# RANDOM IMAGE/CAPTION PAIR
img_id, cap_str = coco.random_pair()
print(img_id, cap_str)

285342 Close up view of some kind of meat with black eyed peas and vegetables.


In [5]:
# VECTORIZING IMAGES
img_dvector = coco.vectorize_image(img_id)
print(img_dvector)

[[8.52659881e-01 2.44662285e-01 4.68955003e-03 9.97656345e-01
  4.48034406e-01 1.04226187e-01 1.09769598e-01 1.72474837e+00
  6.13684841e-02 3.39598179e-01 1.16680288e+00 0.00000000e+00
  9.89537597e-01 4.56126511e-01 9.80125368e-01 2.36543894e-01
  1.58537340e+00 1.12955976e+00 2.16912460e+00 7.98378289e-02
  5.43035030e-01 3.18641007e-01 3.24012011e-01 5.15243292e-01
  1.97668564e+00 1.11735791e-01 2.66474628e+00 3.80649209e-01
  4.31259036e-01 1.22607911e+00 2.32709438e-01 1.27601123e+00
  3.26328486e-01 1.18999168e-01 2.98471391e-01 5.24648488e-01
  4.78155017e-01 9.78624046e-01 6.19935244e-02 1.31917453e+00
  9.25579488e-01 1.65383920e-01 3.44611704e-01 1.25879526e-01
  2.80028768e-02 5.50404489e-01 8.61250013e-02 1.96195054e+00
  1.15808356e+00 5.49530447e-01 2.55597681e-01 8.20780098e-01
  3.27801585e+00 8.48169327e-02 1.22917974e+00 6.27175689e-01
  1.23979859e-01 1.15657175e+00 8.64573598e-01 1.19351327e+00
  1.18235362e+00 2.21458942e-01 3.78897712e-02 5.25794804e-01
  6.9393

In [ ]:
# EMBEDDING TEXT
cap_embed = coco.embed_text(cap_str)
print(cap_embed)